In [0]:
%sql
show databases;

databaseName
default
testdb


In [0]:
%sql
use testdb;

In [0]:
%sql
show tables;

database,tableName,isTemporary
testdb,covid_by_contry,false
testdb,stg_farmer_tweet,false
,job_benefits,true
,job_benefits_view,true
,job_salaries,true
,job_salaries_view,true


In [0]:
%fs ls /

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/user/,user/,0,0


In [0]:
%fs ls dbfs:/FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/benefits.csv,benefits.csv,840280,1708117418000
dbfs:/FileStore/tables/cars_data.csv,cars_data.csv,17592,1707944449000
dbfs:/FileStore/tables/covid_country_dataset.csv,covid_country_dataset.csv,14784,1708107294000
dbfs:/FileStore/tables/farmers_protest_tweets.json,farmers_protest_tweets.json,407720271,1708112257000
dbfs:/FileStore/tables/flightLogs/,flightLogs/,0,0
dbfs:/FileStore/tables/salaries.csv,salaries.csv,731493,1708117418000
dbfs:/FileStore/tables/social_media.json,social_media.json,19261,1707937656000
dbfs:/FileStore/tables/tmdb_credits.csv,tmdb_credits.csv,40044293,1708116796000
dbfs:/FileStore/tables/tmdb_movies.csv,tmdb_movies.csv,5698602,1708116793000
dbfs:/FileStore/tables/tradeData/,tradeData/,0,0


## Create Job Benefits View in the Landing Zone 

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW job_benefits_view
   (job_id bigint ,inferred int,type string)
USING CSV
OPTIONS (
  path = "dbfs:/FileStore/tables/benefits.csv",
  header = "true",
  delimiter = ","
);

## Create Job Salaries View in the Landing Zone 

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW job_salaries_view
   (salary_id int,job_id bigint,max_salary double,med_salary double,min_salary double,pay_period string,currency string,compensation_type string)
USING CSV
OPTIONS (
  path = "dbfs:/FileStore/tables/salaries.csv",
  header = "true",
  delimiter = ","
);

In [0]:
%sql
show tables;

database,tableName,isTemporary
default,covid_by_contry,false
,job_benefits,true
,job_benefits_view,true
,job_salaries,true
,job_salaries_view,true


In [0]:
%sql
select * from job_benefits limit 10;

job_id,inferred,type
3690843087,0,Medical insurance
3690843087,0,Dental insurance
3690843087,0,401(k)
3690843087,0,Paid maternity leave
3690843087,0,Disability insurance
3690843087,0,Vision insurance
3691763971,1,Dental insurance
3691763971,1,Disability insurance
3691763971,1,401(k)
3691775263,0,Medical insurance


In [0]:
%sql
select * from job_salaries where med_salary is not null limit 10;

salary_id,job_id,max_salary,med_salary,min_salary,pay_period,currency,compensation_type
55,3696987686,null,17.3,null,HOURLY,USD,BASE_SALARY
58,3696990991,null,4000.0,null,MONTHLY,USD,BASE_SALARY
59,3696990992,null,4000.0,null,MONTHLY,USD,BASE_SALARY
60,3696990993,null,4000.0,null,MONTHLY,USD,BASE_SALARY
61,3696993074,null,17.3,null,HOURLY,USD,BASE_SALARY
62,3696994822,null,4000.0,null,MONTHLY,USD,BASE_SALARY
63,3696994861,null,4000.0,null,MONTHLY,USD,BASE_SALARY
64,3696995466,null,4000.0,null,MONTHLY,USD,BASE_SALARY
65,3696997069,null,4000.0,null,MONTHLY,USD,BASE_SALARY
66,3696997071,null,4000.0,null,MONTHLY,USD,BASE_SALARY


In [0]:
%sql
select count(*) from job_salaries;

count(1)
13352


In [0]:
%sql
select count(*) from job_benefits;

count(1)
29325


In [0]:
%sql
select count(*) from job_salaries
where med_salary is not null

count(1)
2241


## Create Job Salaries Table in th Staging Zone and limit the count with med salary coloumn

In [0]:
%sql
create table job_salaries_stg
as 
select * from job_salaries
where med_salary is not null

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from job_benefits limit 1500;

job_id,inferred,type
3690843087,0,Medical insurance
3690843087,0,Dental insurance
3690843087,0,401(k)
3690843087,0,Paid maternity leave
3690843087,0,Disability insurance
3690843087,0,Vision insurance
3691763971,1,Dental insurance
3691763971,1,Disability insurance
3691763971,1,401(k)
3691775263,0,Medical insurance


## Create Job Benefits Table in th Staging Zone and limit the record count

In [0]:
%sql
create table job_benefits_stg
as 
select * from job_benefits limit 1500;

num_affected_rows,num_inserted_rows


In [0]:
%sql
show tables;

database,tableName,isTemporary
testdb,covid_by_contry,false
testdb,job_benefits_stg,false
testdb,job_salaries_stg,false
testdb,stg_farmer_tweet,false
,job_benefits,true
,job_benefits_view,true
,job_salaries,true
,job_salaries_view,true


In [0]:
%sql
select * from job_benefits_stg limit 20;

job_id,inferred,type
3690843087,0,Medical insurance
3690843087,0,Dental insurance
3690843087,0,401(k)
3690843087,0,Paid maternity leave
3690843087,0,Disability insurance
3690843087,0,Vision insurance
3691763971,1,Dental insurance
3691763971,1,Disability insurance
3691763971,1,401(k)
3691775263,0,Medical insurance


## Inner Join Operation

In [0]:
%sql
select bs.job_id, bs.type, ss.salary_id, ss.min_salary, ss.max_salary, ss.pay_period, ss.compensation_type from job_salaries_stg ss
inner join job_benefits_stg bs
on ss.job_id = bs.job_id;

job_id,type,salary_id,min_salary,max_salary,pay_period,compensation_type
3696987686,401(k),55,null,null,HOURLY,BASE_SALARY
3696990991,401(k),58,null,null,MONTHLY,BASE_SALARY
3696990992,401(k),59,null,null,MONTHLY,BASE_SALARY
3696990993,401(k),60,null,null,MONTHLY,BASE_SALARY
3696993074,401(k),61,null,null,HOURLY,BASE_SALARY
3696994822,401(k),62,null,null,MONTHLY,BASE_SALARY
3696994861,401(k),63,null,null,MONTHLY,BASE_SALARY
3696995466,401(k),64,null,null,MONTHLY,BASE_SALARY
3696997069,401(k),65,null,null,MONTHLY,BASE_SALARY
3696997071,401(k),66,null,null,MONTHLY,BASE_SALARY


## Left Join Operation

In [0]:
%sql
select bs.job_id, bs.type, ss.salary_id, ss.min_salary, ss.max_salary, ss.pay_period, ss.compensation_type from job_salaries_stg ss
left join job_benefits_stg bs
on ss.job_id = bs.job_id where bs.job_id is not null;

job_id,type,salary_id,min_salary,max_salary,pay_period,compensation_type
3696987686,401(k),55,null,null,HOURLY,BASE_SALARY
3696990991,401(k),58,null,null,MONTHLY,BASE_SALARY
3696990992,401(k),59,null,null,MONTHLY,BASE_SALARY
3696990993,401(k),60,null,null,MONTHLY,BASE_SALARY
3696993074,401(k),61,null,null,HOURLY,BASE_SALARY
3696994822,401(k),62,null,null,MONTHLY,BASE_SALARY
3696994861,401(k),63,null,null,MONTHLY,BASE_SALARY
3696995466,401(k),64,null,null,MONTHLY,BASE_SALARY
3696997069,401(k),65,null,null,MONTHLY,BASE_SALARY
3696997071,401(k),66,null,null,MONTHLY,BASE_SALARY


## Right Join Operation

In [0]:
%sql
select bs.job_id, bs.type, ss.salary_id, ss.min_salary, ss.max_salary, ss.pay_period, ss.compensation_type from job_salaries_stg ss
right join job_benefits_stg bs
on ss.job_id = bs.job_id where bs.job_id is not null;

job_id,type,salary_id,min_salary,max_salary,pay_period,compensation_type
3690843087,Medical insurance,null,null,null,null,null
3690843087,Dental insurance,null,null,null,null,null
3690843087,401(k),null,null,null,null,null
3690843087,Paid maternity leave,null,null,null,null,null
3690843087,Disability insurance,null,null,null,null,null
3690843087,Vision insurance,null,null,null,null,null
3691763971,Dental insurance,null,null,null,null,null
3691763971,Disability insurance,null,null,null,null,null
3691763971,401(k),null,null,null,null,null
3691775263,Medical insurance,null,null,null,null,null
